In [10]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI

In [11]:
load_dotenv(override=True)

True

In [12]:
system_prompt = """
You are an expert tic tac toe {role}. The board is 3x3. The players are X and O. 
    The board is represented by Json array. Where "." represents empty cell. 
    Sample empty board:
    {
        "board": [
            ['.', '.', '.'],
            ['.', '.', '.'],
            ['.', '.', '.']
            ]
    }
    The first player to get 3 of their marks in a row, column, or diagonal wins. 
    Few examples where X is the winner:
    {
        "board": [
            ['X', 'O', '.'],
            ['X', 'O', '.'],
            ['X', '.', '.']
            ]
    }
    {
        "board": [
            ['X', 'O', '.'],
            ['O', 'X', '.'],
            ['.', '.', 'X']
            ]
    }
    Few Examples where O is the winner:
    {
        "board": [
            ['X', 'O', '.'],
            ['X', 'O', '.'],
            ['.', 'O', 'X']
            ]
    }
    {
        "board": [
            ['X', 'O', '.'],
            ['.', 'O', 'X'],
            ['.', 'O', 'X']
        ]
    }
    Few Examples where these is no winner yet:
    {
        "board": [
            ['X', 'O', 'O'],
            ['O', 'X', 'X'],
            ['O', '.', '.']
        ]
    }
    "board": [
        ["X", "O", "O"],
        [".", "O", "."],
        ["X", "X", "O"]
    ]
}
    If all cells are filled and no player has won, the game is a draw.
    Give the result of the game in the same format as the board. Dont respond with anything else. Just the updated state of the board.
"""

def check_winner(board="""{
        "board": [
            ['.', '.', '.'],
            ['.', '.', '.'],
            ['.', '.', '.']
            ]
    }"""):
    return f"""If someone has won respond with on X or O if someone has won, otherwise return "No Winner", just the string "No Winner" and nothing else.
    Did someone win? Respond with on X or O if someone has won, otherwise return "No Winner", just the string "No Winner" and nothing else.
    This is the current state of the board: {board}"""

def next_move(board="""{
        "board": [
            ['.', '.', '.'],
            ['.', '.', '.'],
            ['.', '.', '.']
            ]
    }
              """):
    return f"""Please suggest a move for the next player.MAke sure you move on one of the unoccupied cells represented by "." 
    This is the current state of the board: {board}"""

In [14]:
# client = (OpenAI(), "gpt-4o-mini") 
# client = (OpenAI(api_key=os.getenv("GOOGLE_API_KEY"), base_url="https://generativelanguage.googleapis.com/v1beta/openai/"), "gemini-2.0-flash")
client = (OpenAI(api_key=os.getenv("DEEPSEEK_API_KEY"), base_url="https://api.deepseek.com/v1"), "deepseek-chat")
currentWinner = "No Winner"
currentState = """
    {
        "board": [
            ['.', '.', '.'],
            ['.', '.', '.'],
            ['.', '.', '.']
            ]
    }
"""
def send_message(message):
    completion = client[0].chat.completions.create(
    model=client[1],
    messages=message
    )
    return completion.choices[0].message.content



while currentWinner.__contains__("No Winner"):
    messageNextMove = [{"role": "system", "content": system_prompt}] +  [{"role": "user", "content": next_move(currentState)}]
    messageCheckWinner = [{"role": "system", "content": system_prompt}] +  [{"role": "user", "content": check_winner(currentState)}]
    result=send_message(messageCheckWinner)
    print(f"result: {result}")
    if not result.__contains__("No Winner"):
        currentWinner = result
        print(f"Winner is {currentWinner}")
    else:
        currentState=send_message(messageNextMove)
        print(f"""currentState: {currentState}""")

result: No Winner
currentState: {
    "board": [
        ["X", ".", "."],
        [".", ".", "."],
        [".", ".", "."]
    ]
}
result: No Winner
currentState: {
    "board": [
        ["X", ".", "."],
        [".", "O", "."],
        [".", ".", "."]
    ]
}
result: No Winner
currentState: {
    "board": [
        ["X", ".", "."],
        [".", "O", "."],
        [".", ".", "X"]
    ]
}
result: No Winner
currentState: {
    "board": [
        ["X", "O", "."],
        [".", "O", "."],
        [".", ".", "X"]
    ]
}
result: No Winner
currentState: {
    "board": [
        ["X", "O", "."],
        [".", "O", "."],
        ["X", ".", "X"]
    ]
}
result: No Winner
currentState: {
    "board": [
        ["X", "O", "."],
        [".", "O", "."],
        ["X", "O", "X"]
    ]
}
result: O
Winner is O
